# Chocolate Data SVM w/ Anova  
## LF_HS vs. h2O  
Step by step description of code for the process of decoding the Chocolate Milkshake data, looking at Low Fat(LF) and High Sugar(HS) against water(h2O), using a Support Vector Machine(SVM) with an ANOVA. 

In [ ]:
# Import packages
import os
import numpy as np
import nilearn
import glob
import nibabel as nib
import pandas as pd 
import numpy as np
import warnings
import matplotlib
from sklearn.model_selection import cross_val_score
from nilearn.input_data import NiftiMasker 


matplotlib.use('Agg')
warnings.filterwarnings("ignore")
np.seterr(divide='ignore', invalid='ignore')

### Load Data

In [ ]:
# image mask
imag_mask='/projects/niblab/bids_projects/Experiments/ChocoData/images/bin_mask.nii.gz'

# load the behavioral data
stim = '/projects/niblab/bids_projects/Experiments/ChocoData/behavorial_data/4w_part1.csv'

# concatenated Nifti image of subjects
dataset='/projects/niblab/bids_projects/Experiments/ChocoData/images/4w_part1.nii.gz'

# Prepare data 
# load behavioral data into a pandas dataframe(df)
behavioral = pd.read_csv(stim, sep="\t")
# grab conditional labels 
y = behavioral["Label"]


#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['LF_HS_receipt', "h20_receipt"])
y = y[condition_mask]
#confirm we have the # of condtions needed
print(y.unique())


### Load data in NiftiMasker

In [ ]:
# create masker object passing in our image mask
masker = NiftiMasker(mask_img=imag_mask, standardize=True, memory="nilearn_cache", memory_level=1)
# fit our Nifti image with the masker
X = masker.fit_transform(dataset)
# Apply our condition_mask of target labels
X = X[condition_mask]


### Set up the SVM model with a linear kernel

In [ ]:
# load the support vector classifier, set a linear kernel
from sklearn.svm import SVC
svc = SVC(kernel='linear')

# restrict feature selection to the significant 500 voxels
from sklearn.feature_selection import SelectKBest, f_classif
feature_selection = SelectKBest(f_classif, k=500)


### Setup GridSearchCV pipeline and run nested cross validation(CV)

In [ ]:

nested_cv_scores = cross_val_score(anova_svc, X, y, cv=5)

# print results
print("Nested CV score: %.4f" % np.mean(nested_cv_scores))

### Visualize Results

In [ ]:
# Get coefficient image
coef = svc.coef_
# reverse feature selection
coef = feature_selection.inverse_transform(coef)
# reverse masking
weight_img = masker.inverse_transform(coef)


# Use the mean image as a background to avoid relying on anatomical data
from nilearn import image
mean_img = image.mean_img(dataset)
mean_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/LF_HS_vs_h2O/images/4wp1_mean_nimask.nii')

# Create the figure
from nilearn.plotting import plot_stat_map, show
display = plot_stat_map(weight_img, mean_img, title='SVM weights LF_HS vs h2O 4 waves part 1')
display.savefig('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/LF_HS_vs_h2O/images/4wp1_SVM_nimask.png')
# Saving the results as a Nifti file may also be important
weight_img.to_filename('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/code/decoding/LF_HS_vs_h2O/images/4wp1_SVM_nimask.nii')



Reference: http://nilearn.github.io/auto_examples/02_decoding/plot_haxby_anova_svm.html#sphx-glr-auto-examples-02-decoding-plot-haxby-anova-svm-py